# ทดสอบการเข้ารหัสคอลัมน์ด้วย ORM (SQLAlchemy) + pgcrypto

โน้ตบุ๊กนี้สาธิตการใช้โมเดลฐานข้อมูล (ORM) เพื่อ
- บันทึกข้อมูลเป็นข้อความปกติ แต่เข้ารหัสที่ฝั่งฐานข้อมูล (server-side) ด้วย `pgcrypto`
- อ่านข้อมูลกลับมาเป็นข้อความที่ถอดรหัสแล้ว โดยโปร่งใสผ่านชนิดข้อมูลกำหนดเอง (TypeDecorator)

ข้อดี: ไม่ต้องเขียน SQL เข้ารหัส/ถอดรหัสเองทุกครั้ง — ORM จัดการให้


## เตรียมสภาพแวดล้อม
- รัน `docker-compose up -d` เพื่อเริ่มฐานข้อมูลที่เปิด `pgcrypto` แล้ว
- แนะนำให้เปิดใช้งาน venv และติดตั้งไลบรารีจาก `requirements.txt`
- ถ้ายังไม่ได้ติดตั้ง สามารถใช้เซลล์ด้านล่างนี้ช่วยติดตั้งแบบชั่วคราว

In [1]:
# ติดตั้งไลบรารีที่จำเป็น (ข้ามได้ถ้าติดตั้งแล้ว)
# !pip install -q "psycopg[binary]>=3.1" SQLAlchemy>=2.0 pandas

## ตั้งค่าการเชื่อมต่อ และประกาศโมเดล ORM
- ใช้ `TypeDecorator` เพื่อทำให้คอลัมน์ string เข้ารหัสก่อนบันทึก (bind_expression)
- และถอดรหัสอัตโนมัติเมื่ออ่าน (column_expression) ด้วยฟังก์ชัน `pgp_sym_encrypt/pgp_sym_decrypt`

In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine, Text, LargeBinary, func, text, select, type_coerce
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, Session
from sqlalchemy.types import TypeDecorator

# ค่าการเชื่อมต่อฐานข้อมูล (ปรับแก้ได้ผ่าน ENV)
PG_HOST = os.getenv('PG_HOST', 'localhost')
PG_PORT = int(os.getenv('PG_PORT', '5432'))
PG_DB   = os.getenv('PG_DB', 'encdb')
PG_USER = os.getenv('PG_USER', 'encuser')
PG_PWD  = os.getenv('PG_PASSWORD', 'encpass')
PASSPHRASE = os.getenv('PG_PASSPHRASE', 'my-strong-demo-passphrase')  # ใช้เพื่อการสาธิต

# สร้าง engine ด้วยไดรเวอร์ psycopg3
engine = create_engine(
    f'postgresql+psycopg://{PG_USER}:{PG_PWD}@{PG_HOST}:{PG_PORT}/{PG_DB}',
    future=True,
)

# ให้แน่ใจว่า pgcrypto ถูกเปิดใช้งาน (กรณีฐานข้อมูลใหม่)
with engine.begin() as conn:
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS pgcrypto;"))

# ประกาศ Base สำหรับ ORM
class Base(DeclarativeBase):
    pass

# สร้างชนิดข้อมูลกำหนดเองสำหรับคอลัมน์ที่ต้องเข้ารหัส/ถอดรหัสอัตโนมัติ
class PgSymEncryptedString(TypeDecorator):
    impl = LargeBinary  # เก็บจริงเป็น bytea
    cache_ok = True

    def __init__(self, passphrase: str):
        super().__init__()
        self.passphrase = passphrase

    # แปลงค่าที่จะ bind เข้าไปเป็นคำสั่งเข้ารหัสที่ฝั่งฐานข้อมูล
    def bind_expression(self, bindvalue):
        # ใช้ type_coerce เพื่อให้พารามิเตอร์ถูกมองเป็น TEXT ตั้งแต่ชั้น SQLAlchemy
        # หลีกเลี่ยงปัญหาแคสต์จาก bytea -> text ฝั่งฐานข้อมูล
        return func.pgp_sym_encrypt(type_coerce(bindvalue, Text()), self.passphrase)

    # แปลงคอลัมน์ที่อ่านออกมาให้ถอดรหัสอัตโนมัติ
    def column_expression(self, col):
        # pgp_sym_decrypt คืนค่าเป็น TEXT อยู่แล้ว ใช้ type_coerce เพื่อบอกชนิดกลับไปยัง ORM
        return type_coerce(func.pgp_sym_decrypt(col, self.passphrase), Text())

# โมเดลตัวอย่าง: customer_secret_orm
class CustomerSecretORM(Base):
    __tablename__ = 'customer_secret_orm'
    id: Mapped[int] = mapped_column(primary_key=True, autoincrement=True)
    full_name: Mapped[str] = mapped_column(Text, nullable=False)
    # national_id จะถูกเข้ารหัสก่อนบันทึก และถูกถอดรหัสเมื่ออ่านโดยอัตโนมัติ
    national_id: Mapped[str] = mapped_column(PgSymEncryptedString(PASSPHRASE), nullable=False)

    def __repr__(self) -> str:
        return f"<CustomerSecretORM id={self.id} full_name={self.full_name!r}>"

# สร้างตาราง (ใช้ชื่อใหม่ต่างจากตัวอย่าง SQL ดิบ เพื่อหลีกเลี่ยงชนกัน)
with engine.begin() as conn:
    conn.execute(text('DROP TABLE IF EXISTS customer_secret_orm'))
Base.metadata.create_all(engine)
print('สร้างตาราง customer_secret_orm สำเร็จ')


สร้างตาราง customer_secret_orm สำเร็จ


## แทรกข้อมูลด้วย ORM (จะเข้ารหัสอัตโนมัติที่ฝั่งฐานข้อมูล)

In [3]:
samples = [
    ('สมชาย ใจดี', '1101700200011'),
    ('สมหญิง แข็งแรง', '1101700200022'),
    ('สายฝน สายชล', '1101700200033'),
]

with Session(engine) as session:
    session.add_all([CustomerSecretORM(full_name=n, national_id=i) for n, i in samples])
    session.commit()
print('เพิ่มข้อมูลสำเร็จ (ถูกเข้ารหัสก่อนบันทึก)')


เพิ่มข้อมูลสำเร็จ (ถูกเข้ารหัสก่อนบันทึก)


## อ่านข้อมูลแบบเข้ารหัส (ยังไม่ถอดรหัส)

In [4]:
# เลือกคอลัมน์ bytea จริงจากตารางโดยตรง เพื่อดู ciphertext
with Session(engine) as session:
    result = session.execute(
        select(
            CustomerSecretORM.id,
            CustomerSecretORM.full_name,
            CustomerSecretORM.__table__.c.national_id.label('national_id_encrypted'),
            func.encode(CustomerSecretORM.__table__.c.national_id, 'hex').label('ct_hex'),
            func.octet_length(CustomerSecretORM.__table__.c.national_id).label('ct_bytes')
        ).order_by(CustomerSecretORM.id)
    ).all()

df = pd.DataFrame(result, columns=['id', 'full_name', 'national_id_encrypted', 'ct_hex', 'ct_bytes'])
df


,id,full_name,national_id_encrypted,ct_hex,ct_bytes
0,1,สมชาย ใจดี,1101700200011,c30d04070302519a42b6349ed82c78d23e018237802dda...,79
1,2,สมหญิง แข็งแรง,1101700200022,c30d04070302fdd807de1358174665d23e011717489646...,79
2,3,สายฝน สายชล,1101700200033,c30d04070302abb41e102b738b1d7fd23e015f8e132c9a...,79


## อ่านข้อมูลด้วย ORM (ถอดรหัสอัตโนมัติผ่าน column_expression)

In [5]:
with Session(engine) as session:
    result = session.execute(
        select(CustomerSecretORM.id, CustomerSecretORM.full_name, CustomerSecretORM.national_id)
        .order_by(CustomerSecretORM.id)
    ).all()

df = pd.DataFrame(result, columns=['id', 'full_name', 'national_id'])
df


,id,full_name,national_id
0,1,สมชาย ใจดี,1101700200011
1,2,สมหญิง แข็งแรง,1101700200022
2,3,สายฝน สายชล,1101700200033


## ตรวจสอบข้อมูลที่เก็บจริงเป็น bytea (ยังเป็นข้อมูลเข้ารหัส)
- ด้านล่างนี้คิวรีแบบ SQL ดิบเพื่อดูความยาว ciphertext
- จะเห็นว่าคอลัมน์ในฐานข้อมูลเป็น `bytea` ไม่ใช่ string ธรรมดา

In [6]:
with engine.begin() as conn:
    rows = conn.execute(
        text("SELECT id, full_name, octet_length(national_id) AS ct_bytes FROM customer_secret_orm ORDER BY id")
    ).fetchall()
pd.DataFrame(rows, columns=['id', 'full_name', 'ct_bytes'])


,id,full_name,ct_bytes
0,1,สมชาย ใจดี,79
1,2,สมหญิง แข็งแรง,79
2,3,สายฝน สายชล,79


### หมายเหตุด้านความปลอดภัย
- ตัวอย่างนี้ใช้ passphrase แบบกำหนดใน ENV/ตัวแปร เพื่อความง่ายในการสาธิต
- ในระบบจริงควรเก็บคีย์ใน secret manager และจำกัดสิทธิ์การเข้าถึง
- การเข้ารหัสแบบ symmetric เหมาะกับข้อมูลที่ต้องปกป้องขณะพักเก็บ (at rest)
- การทำ index บนข้อมูลที่เข้ารหัสโดยตรงไม่สามารถทำได้ตามปกติ
- พิจารณาใช้แนวทางอื่น (เช่น แฮชสำหรับค้นหาเท่ากัน) หากมีความต้องการค้นหา
